Wykorzystamy zbiór danych pingwinów zebrany przez stację Palmer na Antarktydzie. Zestaw składa się z danych dotyczących 344 pingwinów znalezionych na trzech wyspach Archipelagu Palmera na końcu Półwyspu Antarktycznego. Na każdym pingwinie wykonuje się 4 pomiary numeryczne, podaje się jego gatunek, płeć i lokalizację. Zbiór danych zawiera 3 gatunki pingwinów i 11 wierszy brakujących danych.

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

import seaborn as sns

numeric = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

penguins = sns.load_dataset('penguins').dropna()

penguins




,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


1. Wygeneruj etykiety numeryczne dla kolumn "species", "island" i "sex".

In [13]:
encoders = {}

for col in ["species", "island", "sex"]:
    le = LabelEncoder()
    penguins[col] = le.fit_transform(penguins[col])
    encoders[col] = le

penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,1
1,0,2,39.5,17.4,186.0,3800.0,0
2,0,2,40.3,18.0,195.0,3250.0,0
4,0,2,36.7,19.3,193.0,3450.0,0
5,0,2,39.3,20.6,190.0,3650.0,1
...,...,...,...,...,...,...,...
338,2,0,47.2,13.7,214.0,4925.0,0
340,2,0,46.8,14.3,215.0,4850.0,0
341,2,0,50.4,15.7,222.0,5750.0,1
342,2,0,45.2,14.8,212.0,5200.0,0


2. Zbuduj (kNN) klasyfikator płci pingwina. - Podziel zbiór danych na zbiór uczący i zestaw testowy i porównaj klasyfikator  ze zbiorem testowym. (accuracy, F1-score) Liczba sąsiadów w KNeighborsClassifier sklearn jest określana za pomocą parametrów n_neighbors. Użyj n_neighbors = 3.

In [14]:
penguins_copy = penguins.copy()
Y = penguins.pop("sex")
X = penguins

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

result = pd.DataFrame([["KNeighborsClassifier",accuracy, f1]], columns=["Model","Accuracy", "F1"])

result


,Model,Accuracy,F1
0,KNeighborsClassifier,0.80597,0.793651


3. Znajdź dobrą wartość dla n_neighbors w klasyfikatorze płci pingwina. - Uruchom szkolenie klasyfikatora w pętli i wydrukuj wynik dokładności dla różnych wartości n_neighbors =. Wartości do n_neighbors = 20 powinny wystarczyć.

In [15]:
rows = []

for n in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    rows.append({"n_neighbors": n,
                 "accuracy": accuracy_score(y_test, y_pred),
                 "f1": f1_score(y_test, y_pred)})

tuning_sex = pd.DataFrame(rows)
tuning_sex

,n_neighbors,accuracy,f1
0,1,0.850746,0.838710
1,2,0.805970,0.763636
2,3,0.805970,0.793651
3,4,0.791045,0.750000
4,5,0.776119,0.754098
5,6,0.791045,0.758621
6,7,0.820896,0.812500
7,8,0.835821,0.813559
8,9,0.820896,0.800000
9,10,0.820896,0.793103


4. Zbuduj klasyfikator gatunków pingwinów i znajdź dobrą wartość dla n_neighbors w tym klasyfikatorze.

In [16]:
X_species = penguins.drop(columns=["species"])
Y_species = penguins["species"]

X_train, X_test, y_train, y_test = train_test_split(
    X_species, Y_species, test_size=0.2, random_state=42, stratify=Y_species
)

rows = []
for n in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    rows.append({
        "n_neighbors": n,
        "accuracy": round(accuracy_score(y_test, y_pred),3),
        "f1_macro": round(f1_score(y_test, y_pred, average="macro"),3)
    })
tuning_species = pd.DataFrame(rows)
tuning_species

,n_neighbors,accuracy,f1_macro
0,1,0.806,0.778
1,2,0.746,0.705
2,3,0.761,0.738
3,4,0.687,0.610
4,5,0.731,0.646
5,6,0.731,0.621
6,7,0.761,0.689
7,8,0.791,0.716
8,9,0.791,0.716
9,10,0.791,0.712
